<a href="https://colab.research.google.com/github/danny6623/Proyecto-Programacion-DB/blob/main/Proyecto_programacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# 📦 INSTALAR DEPENDENCIAS Y CONFIGURAR NGROK
!pip install flask pandas scikit-learn plotly pyngrok -q

# Configurar ngrok con tu authtoken
from pyngrok import ngrok, conf
import os

# Tu authtoken (reemplázalo si es diferente)
ngrok.set_auth_token("38z1pf3mleZMXd17USQXkchZVwa_7F8KYxrtaibDMrrUapKQ1")

print("✅ Ngrok configurado correctamente")

✅ Ngrok configurado correctamente


In [48]:
# 📁 CARGAR DATASET MANUALMENTE
from google.colab import files
import pandas as pd
import io

print("📤 POR FAVOR, SUBE EL ARCHIVO 'ENB2012_data.csv'")
print("=" * 60)
print("1. Haz clic en 'Browse' o 'Elegir archivos'")
print("2. Selecciona el archivo ENB2012_data.csv")
print("3. Espera a que se complete la carga")
print("=" * 60)

# Esta línea abrirá el selector de archivos
uploaded = files.upload()

# Verificar que se subió el archivo
if not uploaded:
    print("\n❌ No se subió ningún archivo. Por favor ejecuta esta celda de nuevo.")
else:
    # Leer el primer archivo subido
    file_name = list(uploaded.keys())[0]
    print(f"\n✅ Archivo '{file_name}' subido exitosamente")

    # Leer el CSV
    df = pd.read_csv(io.BytesIO(uploaded[file_name]))

    print("\n📊 RESUMEN DEL DATASET:")
    print(f"   • Filas: {df.shape[0]}")
    print(f"   • Columnas: {df.shape[1]}")
    print(f"   • Columnas disponibles: {list(df.columns)}")

    # Mostrar las primeras filas
    print("\n📋 PRIMERAS 5 FILAS:")
    print(df.head())

    # Guardar el dataset en una variable global
    global uploaded_df
    uploaded_df = df.copy()

    print("\n" + "=" * 60)
    print("✅ DATASET CARGADO Y LISTO PARA PROCESAR!")
    print("=" * 60)

📤 POR FAVOR, SUBE EL ARCHIVO 'ENB2012_data.csv'
1. Haz clic en 'Browse' o 'Elegir archivos'
2. Selecciona el archivo ENB2012_data.csv
3. Espera a que se complete la carga


Saving ENB2012_data.csv to ENB2012_data (4).csv

✅ Archivo 'ENB2012_data (4).csv' subido exitosamente

📊 RESUMEN DEL DATASET:
   • Filas: 768
   • Columnas: 10
   • Columnas disponibles: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'Y1', 'Y2']

📋 PRIMERAS 5 FILAS:
     X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2
0  0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33
1  0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33
2  0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33
3  0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33
4  0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28

✅ DATASET CARGADO Y LISTO PARA PROCESAR!


In [49]:
# 📁 CREAR MÓDULO: preprocessing.py
%%writefile preprocessing.py
"""
Módulo de preprocesamiento de datos
Universidad Nacional de Chimborazo - Actividad 7
"""
import pandas as pd
import numpy as np

class DataPreprocessor:
    def __init__(self, df):
        """Inicializa el preprocesador con un DataFrame."""
        self.df = df.copy()
        self.original_shape = df.shape

    def handle_missing_values(self):
        """Maneja valores nulos rellenando con la media de cada columna."""
        try:
            if self.df.isnull().sum().any():
                null_count = self.df.isnull().sum().sum()
                print(f"   → Encontrados {null_count} valores nulos")
                self.df = self.df.fillna(self.df.mean())
                print("   ✅ Valores nulos manejados")
            return self.df
        except Exception as e:
            raise Exception(f"Error manejando valores nulos: {str(e)}")

    def normalize_data(self, columns):
        """Normaliza columnas numéricas usando normalización min-max."""
        try:
            normalized_cols = 0
            for col in columns:
                if self.df[col].dtype in [np.float64, np.int64]:
                    min_val = self.df[col].min()
                    max_val = self.df[col].max()
                    if max_val > min_val:  # Evitar división por cero
                        self.df[col] = (self.df[col] - min_val) / (max_val - min_val)
                        normalized_cols += 1
            print(f"   ✅ {normalized_cols} columnas normalizadas")
            return self.df
        except Exception as e:
            raise Exception(f"Error normalizando datos: {str(e)}")

    def get_features_target(self, target_col='Y1'):
        """Separa características (X) y variable objetivo (y)."""
        try:
            if target_col not in self.df.columns:
                available_cols = list(self.df.columns)
                raise Exception(f"Columna '{target_col}' no encontrada. Columnas disponibles: {available_cols}")

            X = self.df.drop(columns=[target_col])
            y = self.df[target_col]
            print(f"   ✅ Características: {X.shape[1]} columnas, Target: {target_col}")
            return X, y
        except Exception as e:
            raise Exception(f"Error separando características y objetivo: {str(e)}")

    def preprocess(self, target_col='Y1'):
        """Ejecuta el pipeline completo de preprocesamiento."""
        print("🔧 Iniciando preprocesamiento...")
        print(f"   Dataset original: {self.original_shape[0]} filas, {self.original_shape[1]} columnas")

        # Paso 1: Manejar valores nulos
        self.handle_missing_values()

        # Paso 2: Normalizar características (excepto target)
        features = [col for col in self.df.columns if col != target_col]
        self.normalize_data(features)

        # Paso 3: Separar X e y
        X, y = self.get_features_target(target_col)

        print(f"   Dataset procesado: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print("   ✅ Preprocesamiento completado")

        return X, y, self.df

Overwriting preprocessing.py


In [50]:
# 📁 CREAR MÓDULO: model.py
%%writefile model.py
"""
Módulo de Machine Learning
Universidad Nacional de Chimborazo - Actividad 7
"""
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd

class MLModel:
    def __init__(self):
        """Inicializa el modelo de Machine Learning."""
        self.model = LinearRegression()
        self.is_trained = False
        self.mae = None
        self.r2 = None
        self.X_test = None
        self.y_test = None
        self.y_pred = None
        self.feature_names = None

    def train(self, X, y):
        """Entrena el modelo de regresión lineal."""
        print("🤖 Iniciando entrenamiento del modelo...")
        print(f"   Forma de X: {X.shape}, Forma de y: {y.shape}")

        try:
            # Guardar nombres de características
            self.feature_names = list(X.columns)

            # Dividir datos en entrenamiento y prueba
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            print(f"   Conjunto de entrenamiento: {X_train.shape[0]} muestras")
            print(f"   Conjunto de prueba: {X_test.shape[0]} muestras")

            # Entrenar modelo
            self.model.fit(X_train, y_train)

            # Hacer predicciones
            y_pred = self.model.predict(X_test)

            # Calcular métricas
            self.mae = mean_absolute_error(y_test, y_pred)
            self.r2 = r2_score(y_test, y_pred)

            # Guardar datos para visualización
            self.X_test = X_test
            self.y_test = y_test
            self.y_pred = y_pred
            self.is_trained = True

            print(f"   ✅ Modelo entrenado exitosamente")
            print(f"   📊 Error Absoluto Medio (MAE): {self.mae:.4f}")
            print(f"   📊 Coeficiente de Determinación (R²): {self.r2:.4f}")
            print(f"   💡 Un R² de {self.r2:.2%} indica que el modelo explica el {self.r2:.2%} de la variabilidad")

            return self.mae, self.r2

        except Exception as e:
            raise Exception(f"Error entrenando el modelo: {str(e)}")

    def predict_new(self, new_data):
        """Predice nuevos datos."""
        if not self.is_trained:
            raise Exception("Modelo no entrenado. Primero entrena el modelo llamando a train()")

        try:
            # Verificar que new_data tenga las mismas columnas
            if isinstance(new_data, dict):
                # Convertir diccionario a DataFrame con una fila
                prediction_df = pd.DataFrame([new_data])
            elif isinstance(new_data, list):
                # Lista de diccionarios
                prediction_df = pd.DataFrame(new_data)
            else:
                prediction_df = new_data

            # Asegurar el orden correcto de columnas
            if self.feature_names:
                # Mantener solo las columnas necesarias
                available_cols = [col for col in self.feature_names if col in prediction_df.columns]
                if len(available_cols) != len(self.feature_names):
                    missing = set(self.feature_names) - set(available_cols)
                    raise Exception(f"Columnas faltantes: {missing}")
                prediction_df = prediction_df[available_cols]

            prediction = self.model.predict(prediction_df)
            return prediction

        except Exception as e:
            raise Exception(f"Error haciendo predicción: {str(e)}")

Overwriting model.py


In [51]:
# 📁 CREAR MÓDULO: exceptions.py
%%writefile exceptions.py
"""
Módulo de excepciones personalizadas
Universidad Nacional de Chimborazo - Actividad 7
"""

class InvalidFileError(Exception):
    """Excepción para archivos no válidos."""
    def __init__(self, message="Archivo no válido"):
        super().__init__(message)

class MissingColumnsError(Exception):
    """Excepción para columnas faltantes."""
    def __init__(self, message="Columnas requeridas faltantes"):
        super().__init__(message)

class ModelNotTrainedError(Exception):
    """Excepción cuando el modelo no está entrenado."""
    def __init__(self, message="El modelo no está entrenado"):
        super().__init__(message)

class DatasetNotLoadedError(Exception):
    """Excepción cuando no hay dataset cargado."""
    def __init__(self, message="No hay dataset cargado"):
        super().__init__(message)

Overwriting exceptions.py


In [57]:
# 🌐 APLICACIÓN FLASK PRINCIPAL - app.py (VERSIÓN MEJORADA)
%%writefile app.py
"""
Aplicación Web Flask con Machine Learning
Universidad Nacional de Chimborazo - Actividad Autónoma 7
Programación Modular y Manejo de Excepciones
"""
import pandas as pd
import numpy as np
import io
import json
import sys
import traceback
from datetime import datetime
from flask import Flask, request, jsonify, render_template_string, Response

# Importar nuestros módulos
try:
    from preprocessing import DataPreprocessor
    from model import MLModel
    from exceptions import *
    print("✅ Módulos importados correctamente")
except ImportError as e:
    print(f"❌ Error importando módulos: {e}")
    print("Asegúrate de haber ejecutado las celdas anteriores")
    sys.exit(1)

# ========== CONFIGURACIÓN FLASK ==========
app = Flask(__name__)

# ========== VARIABLES GLOBALES ==========
current_df = None
ml_model = MLModel()
preprocessor = None
app_log = []

# ========== FUNCIÓN PARA AÑADIR LOGS ==========
def add_log(message, level="INFO"):
    """Añade un mensaje al log de la aplicación."""
    timestamp = datetime.now().strftime("%H:%M:%S")
    log_entry = f"[{timestamp}] [{level}] {message}"
    app_log.append(log_entry)
    print(log_entry)  # También imprimir en consola
    return log_entry

# ========== INTERFAZ HTML SIMPLIFICADA (CSS INCLUIDO) ==========
HTML_TEMPLATE = '''<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>App ML - UNACH Actividad 7</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Arial', sans-serif;
            background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 15px 35px rgba(0,0,0,0.3);
            overflow: hidden;
        }
        .header {
            background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }
        .header h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
        }
        .header p {
            opacity: 0.9;
            font-size: 1.1em;
        }
        .content {
            padding: 30px;
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
            gap: 25px;
        }
        .card {
            background: #f8f9fa;
            border-radius: 15px;
            padding: 25px;
            border-left: 5px solid #3498db;
            transition: all 0.3s ease;
        }
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 10px 25px rgba(0,0,0,0.1);
        }
        .card h2 {
            color: #2c3e50;
            margin-bottom: 20px;
            display: flex;
            align-items: center;
            gap: 10px;
        }
        .step-number {
            background: #3498db;
            color: white;
            width: 35px;
            height: 35px;
            border-radius: 50%;
            display: inline-flex;
            align-items: center;
            justify-content: center;
            margin-right: 10px;
            font-weight: bold;
        }
        button {
            background: linear-gradient(135deg, #3498db 0%, #2980b9 100%);
            color: white;
            border: none;
            padding: 14px 28px;
            border-radius: 8px;
            cursor: pointer;
            font-size: 16px;
            font-weight: 600;
            margin: 10px 5px;
            transition: all 0.3s ease;
            display: inline-flex;
            align-items: center;
            gap: 10px;
        }
        button:hover:not(:disabled) {
            transform: translateY(-3px);
            box-shadow: 0 7px 20px rgba(52, 152, 219, 0.4);
        }
        button:disabled {
            background: #95a5a6;
            cursor: not-allowed;
            opacity: 0.6;
        }
        input[type="file"] {
            padding: 15px;
            border: 2px dashed #3498db;
            border-radius: 8px;
            margin: 15px 0;
            width: 100%;
        }
        textarea {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-family: monospace;
            font-size: 14px;
            margin: 10px 0;
            resize: vertical;
            min-height: 120px;
        }
        .status-box {
            padding: 15px;
            border-radius: 8px;
            margin: 15px 0;
            display: none;
        }
        .success {
            background: #d4edda;
            color: #155724;
            border-left: 5px solid #28a745;
        }
        .error {
            background: #f8d7da;
            color: #721c24;
            border-left: 5px solid #dc3545;
        }
        .info {
            background: #d1ecf1;
            color: #0c5460;
            border-left: 5px solid #17a2b8;
        }
        .loading {
            display: inline-block;
            width: 20px;
            height: 20px;
            border: 3px solid #f3f3f3;
            border-top: 3px solid #3498db;
            border-radius: 50%;
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .log-output {
            background: #2c3e50;
            color: #ecf0f1;
            padding: 15px;
            border-radius: 8px;
            font-family: 'Courier New', monospace;
            font-size: 13px;
            max-height: 250px;
            overflow-y: auto;
            margin-top: 15px;
            white-space: pre-wrap;
        }
        .data-preview {
            background: #fff8e1;
            padding: 15px;
            border-radius: 8px;
            margin-top: 15px;
            max-height: 200px;
            overflow-y: auto;
        }
        .footer {
            text-align: center;
            padding: 20px;
            color: #7f8c8d;
            border-top: 1px solid #eee;
            background: #f8f9fa;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🧠 Aplicación Web con Machine Learning</h1>
            <p>Universidad Nacional de Chimborazo - Actividad Autónoma 8</p>
            <p>Programación Modular y Manejo de Excepciones</p>
            <p>Danny Basantes</p>
        </div>

        <div class="content">
            <div class="card">
                <h2><span class="step-number">1</span> 📤 Cargar Dataset</h2>
                <p>Sube tu archivo CSV con los datos de ENB2012</p>
                <input type="file" id="csvFile" accept=".csv">
                <button onclick="uploadFile()">
                    <span id="uploadIcon">📁</span> Subir Archivo
                </button>
                <div id="uploadStatus" class="status-box"></div>
                <div id="datasetPreview" class="data-preview"></div>
            </div>

            <div class="card">
                <h2><span class="step-number">2</span> 🔧 Preprocesar Datos</h2>
                <p>Limpieza y normalización de datos</p>
                <button id="preprocessBtn" onclick="preprocessData()" disabled>
                    <span id="preprocessIcon">⚙️</span> Preprocesar
                </button>
                <div id="preprocessStatus" class="status-box"></div>
            </div>

            <div class="card">
                <h2><span class="step-number">3</span> 🤖 Entrenar Modelo</h2>
                <p>Entrenar modelo de Regresión Lineal</p>
                <button id="trainBtn" onclick="trainModel()" disabled>
                    <span id="trainIcon">📚</span> Entrenar Modelo
                </button>
                <div id="trainStatus" class="status-box"></div>
            </div>

            <div class="card">
                <h2><span class="step-number">4</span> 🔮 Hacer Predicción</h2>
                <p>Predice nuevos valores (formato JSON):</p>
                <textarea id="inputData">{"X1":0.85,"X2":600,"X3":300,"X4":150,"X5":5,"X6":3,"X7":0.1,"X8":1}</textarea>
                <button id="predictBtn" onclick="makePrediction()" disabled>
                    <span id="predictIcon">✨</span> Predecir
                </button>
                <div id="predictionStatus" class="status-box"></div>
            </div>

            <div class="card">
                <h2><span class="step-number">5</span> 📊 Visualización</h2>
                <p>Visualiza los resultados del modelo</p>
                <button id="visualizeBtn" onclick="showVisualization()" disabled>
                    <span id="visualizeIcon">📈</span> Ver Gráfico
                </button>
                <button id="downloadBtn" onclick="downloadPlot()" disabled>
                    <span id="downloadIcon">💾</span> Descargar PNG
                </button>
                <div id="visualizationStatus" class="status-box"></div>
            </div>

            <div class="card" style="grid-column: 1 / -1;">
                <h2>📝 Logs del Sistema</h2>
                <div class="log-output" id="systemLog">Esperando acción del usuario...</div>
                <button onclick="clearLogs()" style="background: #e74c3c;">
                    🗑️ Limpiar Logs
                </button>
                <button onclick="refreshApp()" style="background: #2ecc71;">
                    🔄 Reiniciar App
                </button>
            </div>
        </div>

        <div class="footer">
            <p>Desarrollado para la Actividad Autónoma 8 - UNACH</p>
            <p>Ciencia de Datos • Programación Modular • Danny Basantes 2026</p>
        </div>
    </div>

    <script>
    function log(message) {
        const logDiv = document.getElementById('systemLog');
        const timestamp = new Date().toLocaleTimeString();
        const logEntry = `[${timestamp}] ${message}\\n`;
        logDiv.innerHTML += logEntry;
        logDiv.scrollTop = logDiv.scrollHeight;
    }

    function clearLogs() {
        document.getElementById('systemLog').innerHTML = 'Logs limpiados...\\n';
        log('Logs del sistema limpiados');
    }

    function refreshApp() {
        if (confirm('¿Estás seguro de reiniciar la aplicación? Se perderán los datos actuales.')) {
            location.reload();
        }
    }

    function showStatus(elementId, message, type) {
        const element = document.getElementById(elementId);
        element.innerHTML = message;
        element.className = 'status-box ' + type;
        element.style.display = 'block';
        log(message.replace(/<[^>]*>/g, ''));
    }

    function uploadFile() {
        const fileInput = document.getElementById('csvFile');
        if (!fileInput.files[0]) {
            showStatus('uploadStatus', '❌ Por favor selecciona un archivo', 'error');
            return;
        }

        const uploadIcon = document.getElementById('uploadIcon');
        uploadIcon.innerHTML = '⏳';
        showStatus('uploadStatus', '<div class="loading"></div> Subiendo archivo...', 'info');

        const formData = new FormData();
        formData.append('file', fileInput.files[0]);

        fetch('/upload', { method: 'POST', body: formData })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    showStatus('uploadStatus', `❌ ${data.error}`, 'error');
                    uploadIcon.innerHTML = '📁';
                } else {
                    showStatus('uploadStatus',
                        `✅ ${data.message}<br>
                         📊 ${data.rows} filas, ${data.cols} columnas<br>
                         📋 Columnas: ${data.columns.join(', ')}`, 'success');

                    document.getElementById('preprocessBtn').disabled = false;
                    document.getElementById('datasetPreview').innerHTML =
                        `<strong>Vista previa (5 filas):</strong><br><pre>${data.preview}</pre>`;
                    uploadIcon.innerHTML = '✅';
                    log(`Dataset cargado: ${data.rows} filas, ${data.cols} columnas`);
                }
            })
            .catch(error => {
                showStatus('uploadStatus', `❌ Error de conexión: ${error}`, 'error');
                uploadIcon.innerHTML = '📁';
            });
    }

    function preprocessData() {
        const icon = document.getElementById('preprocessIcon');
        icon.innerHTML = '⏳';
        showStatus('preprocessStatus', '<div class="loading"></div> Preprocesando datos...', 'info');

        fetch('/preprocess', { method: 'POST' })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    showStatus('preprocessStatus', `❌ ${data.error}`, 'error');
                    icon.innerHTML = '⚙️';
                } else {
                    showStatus('preprocessStatus',
                        `✅ ${data.message}<br>
                         📈 ${data.shape}<br>
                         🔧 ${data.details}`, 'success');

                    document.getElementById('trainBtn').disabled = false;
                    icon.innerHTML = '✅';
                    log('Datos preprocesados correctamente');
                }
            })
            .catch(error => {
                showStatus('preprocessStatus', `❌ Error: ${error}`, 'error');
                icon.innerHTML = '⚙️';
            });
    }

    function trainModel() {
        const icon = document.getElementById('trainIcon');
        icon.innerHTML = '⏳';
        showStatus('trainStatus', '<div class="loading"></div> Entrenando modelo...', 'info');

        fetch('/train', { method: 'POST' })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    showStatus('trainStatus', `❌ ${data.error}`, 'error');
                    icon.innerHTML = '📚';
                } else {
                    showStatus('trainStatus',
                        `✅ ${data.message}<br>
                         📊 MAE: ${data.mae.toFixed(4)}<br>
                         🎯 R²: ${data.r2.toFixed(4)} (${(data.r2*100).toFixed(1)}%)`, 'success');

                    document.getElementById('predictBtn').disabled = false;
                    document.getElementById('visualizeBtn').disabled = false;
                    document.getElementById('downloadBtn').disabled = false;
                    icon.innerHTML = '✅';
                    log(`Modelo entrenado - MAE: ${data.mae.toFixed(4)}, R²: ${data.r2.toFixed(4)}`);
                }
            })
            .catch(error => {
                showStatus('trainStatus', `❌ Error: ${error}`, 'error');
                icon.innerHTML = '📚';
            });
    }

    function makePrediction() {
        const icon = document.getElementById('predictIcon');
        icon.innerHTML = '⏳';
        const inputData = document.getElementById('inputData').value;

        try {
            JSON.parse(inputData);
        } catch (e) {
            showStatus('predictionStatus', '❌ Formato JSON inválido', 'error');
            icon.innerHTML = '✨';
            return;
        }

        showStatus('predictionStatus', '<div class="loading"></div> Calculando predicción...', 'info');

        fetch('/predict', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: inputData
        })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    showStatus('predictionStatus', `❌ ${data.error}`, 'error');
                    icon.innerHTML = '✨';
                } else {
                    showStatus('predictionStatus',
                        `✅ Predicción exitosa!<br>
                         🎯 <strong>${data.prediction}</strong><br>
                         ⏰ ${data.timestamp}`, 'success');
                    icon.innerHTML = '✅';
                    log(`Predicción realizada: ${data.prediction}`);
                }
            })
            .catch(error => {
                showStatus('predictionStatus', `❌ Error: ${error}`, 'error');
                icon.innerHTML = '✨';
            });
    }

    function showVisualization() {
        window.open('/plot', '_blank');
    }

    function downloadPlot() {
        window.open('/download_plot', '_blank');
    }

    // Inicializar logs
    log('Aplicación cargada. Lista para usar.');
    log('Por favor, sube el archivo CSV para comenzar.');
    </script>
</body>
</html>'''

# ========== MANEJADOR DE EXCEPCIONES ==========

@app.errorhandler(Exception)
def handle_exception(e):
    """Maneja todas las excepciones no capturadas."""
    error_msg = f"Error no controlado: {str(e)}"
    add_log(error_msg, "ERROR")
    return jsonify({'error': 'Error interno del servidor. Revisa los logs.'}), 500

# ========== RUTAS FLASK ==========

@app.route('/')
def home():
    """Página principal con la interfaz web."""
    add_log("Página principal cargada", "INFO")
    return render_template_string(HTML_TEMPLATE)

@app.route('/upload', methods=['POST'])
def upload_file():
    """Endpoint para cargar archivo CSV."""
    global current_df
    try:
        if 'file' not in request.files:
            raise InvalidFileError("No se envió ningún archivo")

        file = request.files['file']

        if file.filename == '':
            raise InvalidFileError("No se seleccionó archivo")

        if not file.filename.endswith('.csv'):
            raise InvalidFileError("Solo se permiten archivos CSV (.csv)")

        # Leer el CSV
        content = file.read().decode('utf-8')
        df = pd.read_csv(io.StringIO(content))

        # Validaciones
        if df.empty:
            raise InvalidFileError("El archivo CSV está vacío")

        if len(df.columns) < 2:
            raise InvalidFileError("El CSV debe tener al menos 2 columnas")

        # Guardar dataset
        current_df = df
        preview = df.head().to_string(index=False)

        add_log(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas", "SUCCESS")
        add_log(f"Columnas: {list(df.columns)}", "INFO")

        return jsonify({
            'message': 'Dataset cargado exitosamente',
            'rows': df.shape[0],
            'cols': df.shape[1],
            'columns': list(df.columns),
            'preview': preview
        })

    except InvalidFileError as e:
        add_log(f"Error cargando archivo: {str(e)}", "ERROR")
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        add_log(f"Error inesperado en upload: {str(e)}", "ERROR")
        return jsonify({'error': f'Error procesando archivo: {str(e)}'}), 500

@app.route('/preprocess', methods=['POST'])
def preprocess():
    """Endpoint para preprocesar datos."""
    global current_df, preprocessor
    try:
        if current_df is None:
            raise DatasetNotLoadedError("Primero carga un dataset usando el Paso 1")

        add_log("Iniciando preprocesamiento de datos", "INFO")

        preprocessor = DataPreprocessor(current_df)
        X, y, processed_df = preprocessor.preprocess('Y1')
        current_df = processed_df

        add_log("Preprocesamiento completado", "SUCCESS")

        return jsonify({
            'message': 'Datos preprocesados correctamente',
            'shape': f'{processed_df.shape[0]} filas × {processed_df.shape[1]} columnas',
            'details': f'Target: Y1, Características: {X.shape[1]}'
        })

    except DatasetNotLoadedError as e:
        add_log(f"Error preprocesamiento: {str(e)}", "ERROR")
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        add_log(f"Error en preprocesamiento: {str(e)}", "ERROR")
        return jsonify({'error': f'Error en preprocesamiento: {str(e)}'}), 500

@app.route('/train', methods=['POST'])
def train():
    """Endpoint para entrenar el modelo."""
    global current_df, ml_model, preprocessor
    try:
        if current_df is None:
            raise DatasetNotLoadedError("Primero carga y preprocesa los datos")

        if preprocessor is None:
            preprocessor = DataPreprocessor(current_df)

        add_log("Iniciando entrenamiento del modelo", "INFO")

        X, y, _ = preprocessor.preprocess('Y1')
        mae, r2 = ml_model.train(X, y)

        add_log(f"Modelo entrenado - MAE: {mae:.4f}, R²: {r2:.4f}", "SUCCESS")

        return jsonify({
            'message': 'Modelo entrenado exitosamente',
            'mae': float(mae),
            'r2': float(r2)
        })

    except DatasetNotLoadedError as e:
        add_log(f"Error entrenamiento: {str(e)}", "ERROR")
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        add_log(f"Error entrenando modelo: {str(e)}", "ERROR")
        return jsonify({'error': f'Error al entrenar modelo: {str(e)}'}), 500

@app.route('/predict', methods=['POST'])
def predict():
    """Endpoint para hacer predicciones."""
    global ml_model
    try:
        if not ml_model.is_trained:
            raise ModelNotTrainedError("Primero entrena el modelo (Paso 3)")

        data = request.get_json()

        if not data:
            raise InvalidFileError("No se enviaron datos para predecir")

        add_log("Realizando predicción", "INFO")

        # Convertir a DataFrame
        new_data = pd.DataFrame([data] if isinstance(data, dict) else data)

        # Verificar columnas requeridas
        if ml_model.feature_names:
            missing = set(ml_model.feature_names) - set(new_data.columns)
            if missing:
                raise MissingColumnsError(f"Faltan columnas: {', '.join(missing)}")

        # Hacer predicción
        prediction = ml_model.predict_new(new_data)

        timestamp = datetime.now().strftime("%H:%M:%S")

        add_log(f"Predicción realizada: {prediction[0]:.6f}", "SUCCESS")

        return jsonify({
            'prediction': f'Y1 ≈ {prediction[0]:.6f}' if len(prediction) == 1 else [f'{p:.6f}' for p in prediction],
            'timestamp': timestamp
        })

    except ModelNotTrainedError as e:
        add_log(f"Error predicción: {str(e)}", "ERROR")
        return jsonify({'error': str(e)}), 400
    except MissingColumnsError as e:
        add_log(f"Error predicción: {str(e)}", "ERROR")
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        add_log(f"Error en predicción: {str(e)}", "ERROR")
        return jsonify({'error': f'Error en predicción: {str(e)}'}), 500

@app.route('/plot')
def plot():
    """Endpoint para mostrar gráfico interactivo."""
    try:
        if not ml_model.is_trained:
            return "<h1 style='color: red; padding: 20px;'>⚠️ Primero entrena el modelo (Paso 3)</h1>", 400

        add_log("Generando gráfico interactivo", "INFO")

        import plotly.express as px
        import plotly.graph_objects as go

        # Crear gráfico
        fig = px.scatter(
            x=ml_model.y_test,
            y=ml_model.y_pred,
            labels={'x': 'Valor Real (Y1)', 'y': 'Predicción'},
            title='Modelo de Regresión Lineal - UNACH Actividad 8',
            trendline='ols'
        )

        # Añadir línea ideal
        min_val = min(ml_model.y_test.min(), ml_model.y_pred.min())
        max_val = max(ml_model.y_test.max(), ml_model.y_pred.max())

        fig.add_trace(go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            name='Línea ideal',
            line=dict(dash='dash', color='red')
        ))

        # Añadir métricas
        fig.add_annotation(
            text=f'MAE: {ml_model.mae:.4f}<br>R²: {ml_model.r2:.4f}',
            xref='paper', yref='paper',
            x=0.05, y=0.95,
            showarrow=False,
            bgcolor='white',
            bordercolor='black',
            borderwidth=1
        )

        add_log("Gráfico generado exitosamente", "SUCCESS")

        return fig.to_html(full_html=True, include_plotlyjs='cdn')

    except Exception as e:
        add_log(f"Error generando gráfico: {str(e)}", "ERROR")
        return f"<h1>Error al generar gráfico</h1><pre>{str(e)}</pre>", 500

@app.route('/download_plot')
def download_plot():
    """Endpoint para descargar el gráfico como PNG."""
    try:
        if not ml_model.is_trained:
            return "Primero entrena el modelo", 400

        add_log("Generando gráfico para descarga", "INFO")

        import plotly.io as pio
        import plotly.express as px

        # Crear gráfico simple para descarga
        fig = px.scatter(
            x=ml_model.y_test,
            y=ml_model.y_pred,
            labels={'x': 'Valor Real', 'y': 'Predicción'},
            title='Modelo Regresión Lineal - UNACH'
        )

        # Convertir a imagen
        img_bytes = pio.to_image(fig, format='png', width=800, height=600)

        add_log("Gráfico listo para descarga", "SUCCESS")

        return Response(
            img_bytes,
            mimetype='image/png',
            headers={'Content-Disposition': 'attachment; filename=modelo_unach.png'}
        )

    except Exception as e:
        add_log(f"Error descargando gráfico: {str(e)}", "ERROR")
        return f"Error: {str(e)}", 500

@app.route('/logs')
def get_logs():
    """Endpoint para obtener los logs de la aplicación."""
    return jsonify({'logs': app_log})

# ========== INICIAR SERVIDOR CON NGROK ==========
def run_flask_with_ngrok():
    """Inicia Flask y crea un túnel ngrok."""
    from threading import Thread
    import time

    # Función para ejecutar Flask en un hilo separado
    def run_flask():
        print("🚀 Iniciando servidor Flask en segundo plano...")
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

    # Iniciar Flask en un hilo
    flask_thread = Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()

    # Esperar a que Flask se inicie
    time.sleep(3)

    # Crear túnel ngrok
    print("🌐 Creando túnel ngrok...")
    try:
        # Configurar ngrok
        from pyngrok import ngrok

        # Abrir túnel HTTP
        public_url = ngrok.connect(5000).public_url
        print("\n" + "="*60)
        print("✅ ¡SERVIDOR LISTO!")
        print("="*60)
        print(f"\n🌐 **ENLACE PÚBLICO:** {public_url}")
        print("\n📋 **INSTRUCCIONES:**")
        print("1. Copia el enlace de arriba")
        print("2. Ábrelo en una NUEVA pestaña de tu navegador")
        print("3. Sigue los pasos en la interfaz web")
        print("\n⏳ La aplicación estará disponible por 2 horas")
        print("="*60 + "\n")

        # Mantener el script ejecutándose
        try:
            while True:
                time.sleep(10)
        except KeyboardInterrupt:
            print("\n🔴 Cerrando servidor...")
            ngrok.kill()

    except Exception as e:
        print(f"❌ Error creando túnel ngrok: {e}")
        print("\n💡 SOLUCIÓN ALTERNATIVA:")
        print("1. Ejecuta manualmente: !python app.py")
        print("2. Luego en otra celda ejecuta: !ngrok http 5000")
        print("3. Copia el enlace que genera ngrok")

if __name__ == '__main__':
    print("="*60)
    print("UNACH - ACTIVIDAD AUTÓNOMA 8")
    print("Aplicación Web con Flask y Machine Learning")
    print("="*60)

    run_flask_with_ngrok()

Overwriting app.py


In [56]:
# 🚀 EJECUTAR LA APLICACIÓN COMPLETA
print("🎯 INICIANDO APLICACIÓN COMPLETA...")
print("="*60)
print("Este proceso hará:")
print("1. ✅ Verificar módulos creados")
print("2. 🚀 Iniciar servidor Flask")
print("3. 🌐 Crear túnel público con ngrok")
print("4. 🔗 Generar enlace para acceder desde navegador")
print("="*60)

# Verificar que los archivos se crearon
import os

print("\n📁 VERIFICANDO ARCHIVOS...")
files_to_check = ['preprocessing.py', 'model.py', 'exceptions.py', 'app.py']
all_ok = True

for file in files_to_check:
    if os.path.exists(file):
        print(f"   ✅ {file}")
    else:
        print(f"   ❌ {file} (NO ENCONTRADO)")
        all_ok = False

if not all_ok:
    print("\n⚠️ Algunos archivos no se crearon. Por favor ejecuta las celdas anteriores nuevamente.")
else:
    print("\n✅ Todos los archivos están listos")

    # Verificar si el dataset se cargó
    try:
        if 'uploaded_df' in globals():
            print(f"📊 Dataset cargado: {uploaded_df.shape[0]} filas, {uploaded_df.shape[1]} columnas")
        else:
            print("⚠️ No se detectó dataset cargado. Asegúrate de ejecutar la celda de carga de datos.")
    except:
        pass

    print("\n🚀 EJECUTANDO APLICACIÓN FLASK...")
    print("⏳ Por favor espera unos segundos...")

    # Ejecutar la aplicación
    !python app.py

🎯 INICIANDO APLICACIÓN COMPLETA...
Este proceso hará:
1. ✅ Verificar módulos creados
2. 🚀 Iniciar servidor Flask
3. 🌐 Crear túnel público con ngrok
4. 🔗 Generar enlace para acceder desde navegador

📁 VERIFICANDO ARCHIVOS...
   ✅ preprocessing.py
   ✅ model.py
   ✅ exceptions.py
   ✅ app.py

✅ Todos los archivos están listos
📊 Dataset cargado: 768 filas, 10 columnas

🚀 EJECUTANDO APLICACIÓN FLASK...
⏳ Por favor espera unos segundos...
✅ Módulos importados correctamente
UNACH - ACTIVIDAD AUTÓNOMA 8
Aplicación Web con Flask y Machine Learning
🚀 Iniciando servidor Flask en segundo plano...
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
🌐 Creando túnel ngrok...

✅ ¡SERVIDOR LISTO!

🌐 **ENLACE PÚBLICO:** https://nasopalatine-mariana-miliary.ngrok-free.dev

📋 **INSTRUCCIONES:**
1. Copia el enlace de arriba
2. Ábrelo en una NUEVA pestaña de tu navegador
3. Sigue l